<h1> Creación de Datasets

<h3> Creación de arreglos de NumPy para almacenar los datos de entrenamiento, validación y prueba de ambos modelos

Equipo de Reto 5

Inteligencia artificial avanzada para la ciencia de datos II (Grupo 502)

30 de noviembre de 2023

Modificacion utilizando data augmetation para el 20% del dataset, modificando el brillo de entre 50% a 150% de su brillo original aleatoriamente 

Despues de haber creado el dataset se obtuvieron peores resultados 

### Preparar el entorno de trabajo

In [42]:
# Importar librerias y módulos necesarios
import os
import pandas as pd
import cv2
import numpy as np
import sys
import random
from scipy.ndimage import gaussian_filter

In [43]:
# Definir el directorio de trabajo actual
os.chdir('/home/alanr/Documents/Corazon')

In [44]:
# Definir dimensiones importantes de los datos
img_height = 112
img_width = 112
img_channels = 1
mask_channels = 1
landmarks_channels = 7

### Cargar la lista de archivos y de trazos

In [45]:
# Lista de archivos
file_list = pd.read_csv(os.path.join(os.getcwd(), 'EchoNet-Dynamic/FileList.csv'))
file_list.head()

,FileName,EF,ESV,EDV,FrameHeight,FrameWidth,FPS,NumberOfFrames,Split
0,0X100009310A3BD7FC,78.498406,14.881368,69.210534,112,112,50,174,VAL
1,0X1002E8FBACD08477,59.101988,40.383876,98.742884,112,112,50,215,TRAIN
2,0X1005D03EED19C65B,62.363798,14.267784,37.909734,112,112,50,104,TRAIN
3,0X10075961BC11C88E,54.545097,33.143084,72.914210,112,112,55,122,TRAIN
4,0X10094BA0A028EAC3,24.887742,127.581945,169.855024,112,112,52,207,VAL


In [46]:
# Lista de trazos
volume_tracings = pd.read_csv(os.path.join(os.getcwd(), 'EchoNet-Dynamic/VolumeTracings.csv'))
volume_tracings.head()

,FileName,X1,Y1,X2,Y2,Frame
0,0X100009310A3BD7FC.avi,51.260417,15.348958,64.932292,69.125000,46
1,0X100009310A3BD7FC.avi,50.037611,17.167841,53.367222,16.321330,46
2,0X100009310A3BD7FC.avi,49.157378,20.407629,57.090549,18.390722,46
3,0X100009310A3BD7FC.avi,48.538173,23.581055,59.997339,20.667707,46
4,0X100009310A3BD7FC.avi,47.918968,26.754480,62.904129,22.944693,46


In [47]:
# Identificar todos los fotogramas trazados
labeled_frames = volume_tracings.groupby(['FileName', 'Frame']).size().reset_index(name='Count')
labeled_frames.head()

,FileName,Frame,Count
0,0X100009310A3BD7FC.avi,46,21
1,0X100009310A3BD7FC.avi,61,21
2,0X1002E8FBACD08477.avi,3,21
3,0X1002E8FBACD08477.avi,18,21
4,0X1005D03EED19C65B.avi,24,21


In [48]:
# Borrar del listado los fotogramas de videos que no existen o producen errores
labeled_frames.drop([7761, 8772, 10670, 10671, 11325, 12282, 12509, 18963, 19239, 19699], inplace=True)

### Crear arreglos para almacenar los datos de entrenamiento, validación y prueba

In [49]:
# Conjunto de entrenamiento
train_images = np.empty((0, img_height, img_width, img_channels), dtype=np.float16)
train_masks = np.empty((0, img_height, img_width, mask_channels), dtype=np.float16)
train_landmarks = np.empty((0, img_height, img_width, landmarks_channels), dtype=np.float16)

# Conjunto de validación
val_images = np.empty((0, img_height, img_width, img_channels), dtype=np.float16)
val_masks = np.empty((0, img_height, img_width, mask_channels), dtype=np.float16)
val_landmarks = np.empty((0, img_height, img_width, landmarks_channels), dtype=np.float16)

# Conjunto de prueba
test_images = np.empty((0, img_height, img_width, img_channels), dtype=np.float16)
test_masks = np.empty((0, img_height, img_width, mask_channels), dtype=np.float16)
test_landmarks = np.empty((0, img_height, img_width, landmarks_channels), dtype=np.float16)

### Dividir y guardar los datos de los conjuntos de entrenamiento, validación y prueba

In [50]:
# OPCIONAL: Seleccionar un rango de datos
labeled_frames = labeled_frames.iloc[(labeled_frames.shape[0]//4)*3:]

In [51]:
# variables para data augmentation
train_count = 0
val_count = 0
test_count = 0

In [52]:
def adjust_brightness(image, factor):
    adjusted_image = np.clip(image * factor, 0, 255).astype(np.uint8)
    return adjusted_image

brightness_factor = [0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5 ]



In [53]:
# Para todos los fotogramas trazados
for index, series in labeled_frames.iterrows():

  # Abrir el fotograma
  videos_path = os.path.join(os.getcwd(), 'EchoNet-Dynamic/Videos', series[0])
  cap = cv2.VideoCapture(videos_path)
  cap.set(cv2.CAP_PROP_POS_FRAMES, series[1])
  _, frame = cap.read()
  cap.release()

  # Ordenar los puntos del trazo utilizando el algoritmo Convex Hull
  points = volume_tracings[(volume_tracings['FileName'] == series[0]) & (volume_tracings['Frame'] == series[1])]
  c_h_points = pd.DataFrame({'X': pd.concat([points.X1, points.X2], ignore_index=True), 'Y': pd.concat([points.Y1, points.Y2], ignore_index=True)})
  c_h_points = np.array(c_h_points).astype(np.int32).reshape((-1, 1, 2))

  # Crear una máscara del fotograma
  h, w, _ = frame.shape
  mask = np.zeros((frame.shape[0], frame.shape[1], mask_channels)).astype(np.uint8)
  cv2.fillPoly(mask, [cv2.convexHull(c_h_points)], 255)

  # Almacenar las pendientes y puntos de las líneas trazadas
  slopes, left, right = [], [], []
  for _, line in points.iterrows():
    if line['X2'] == line['X1']:
      slopes.append(sys.maxsize)
      left.append((line['X1'], line['Y1']))
      right.append((line['X2'], line['Y2']))
    elif line['X2'] > line['X1']:
      slopes.append((line['Y2'] - line['Y1']) / (line['X2'] - line['X1']))
      left.append((line['X1'], line['Y1']))
      right.append((line['X2'], line['Y2']))
    else:
      slopes.append((line['Y1'] - line['Y2']) / (line['X1'] - line['X2']))
      left.append((line['X2'], line['Y2']))
      right.append((line['X1'], line['Y1']))

  # Identificar la única línea vertical trazada
  if abs(max(slopes) - np.mean(slopes)) > abs(min(slopes) - np.mean(slopes)):
    a_index = slopes.index(max(slopes))
  else:
    a_index = slopes.index(min(slopes))

  # Guardar la coordenada del punto más alto del trazo
  coords = []
  apex = points.iloc[a_index]
  if apex['Y2'] < apex['Y1']:
    coords.append((apex['X2'], apex['Y2']))
  else:
    coords.append((apex['X1'], apex['Y1']))

  # Guardar la coordenada de los puntos más bajos del trazo
  max_area = 0
  for l in left:
    for r in right:
      area = 0.5 * abs(coords[0][0] * (l[1] - r[1]) + l[0] * (r[1] - coords[0][1]) + r[0] * (coords[0][1] - l[1]))
      if area >= max_area:
        max_area = area
        bases = (l, r)
  coords.append(bases[0])
  coords.append(bases[1])

  # Identificar los puntos intermedios en cada lado
  left = sorted(left, key=lambda l: l[1])
  right = sorted(right, key=lambda r: r[1])
  left = left[:left.index(coords[1])]
  right = right[:right.index(coords[2])]
  coords.append(left[len(left)//3])
  coords.append(right[len(right)//3])
  coords.append(left[2*len(left)//3])
  coords.append(right[2*len(right)//3])

  # Crear un arreglo con los puntos más importantes
  landmarks = np.zeros((frame.shape[0], frame.shape[1], len(coords)), dtype=np.uint8)
  for coord in coords:
    channel_image = np.zeros((frame.shape[:2]))
    cv2.circle(channel_image, (int(coord[0]), int(coord[1])), 4, 255, -1)
    channel_image = gaussian_filter(channel_image, sigma=2)
    landmarks[:,:,coords.index(coord)] = channel_image

  # Guardar el fotograma, la máscara y los puntos más importantes en los arreglos correspondientes
  split = file_list.loc[file_list['FileName'] == series[0][:-4], 'Split'].iloc[0]
  greyscale_frame = np.expand_dims(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), axis=-1).astype(np.uint8)
  if split == 'TRAIN':
      train_count += 1
      if train_count == 6:
        mod_frame = adjust_brightness(greyscale_frame, random.choice(brightness_factor))
        train_images = np.append(train_images.astype(np.float16), [(mod_frame/255.0).astype(np.float16)], axis=0)
        train_masks = np.append(train_masks.astype(np.float16), [(mask/255.0).astype(np.float16)], axis=0)
        train_landmarks = np.append(train_landmarks.astype(np.float16), [(landmarks/255.0).astype(np.float16)], axis=0)
        train_count = 0
      else:
        train_images = np.append(train_images.astype(np.float16), [(greyscale_frame/255.0).astype(np.float16)], axis=0)
        train_masks = np.append(train_masks.astype(np.float16), [(mask/255.0).astype(np.float16)], axis=0)
        train_landmarks = np.append(train_landmarks.astype(np.float16), [(landmarks/255.0).astype(np.float16)], axis=0)

  if split == 'VAL':
      
      val_count += 1
      if val_count == 6:
        mod_frame = adjust_brightness(greyscale_frame, random.choice(brightness_factor))
        val_images = np.append(val_images.astype(np.float16), [(mod_frame/255.0).astype(np.float16)], axis=0)
        val_masks = np.append(val_masks.astype(np.float16), [(mask/255.0).astype(np.float16)], axis=0)
        val_landmarks = np.append(val_landmarks.astype(np.float16), [(landmarks/255.0).astype(np.float16)], axis=0)
        val_count = 0
      else:
        val_images = np.append(val_images.astype(np.float16), [(greyscale_frame/255.0).astype(np.float16)], axis=0)
        val_masks = np.append(val_masks.astype(np.float16), [(mask/255.0).astype(np.float16)], axis=0)
        val_landmarks = np.append(val_landmarks.astype(np.float16), [(landmarks/255.0).astype(np.float16)], axis=0)
  
  if split == 'TEST':
      test_count += 1
      if test_count == 6:
        mod_frame = adjust_brightness(greyscale_frame, random.choice(brightness_factor))
        test_images = np.append(test_images.astype(np.float16), [(mod_frame/255.0).astype(np.float16)], axis=0)
        test_masks = np.append(test_masks.astype(np.float16), [(mask/255.0).astype(np.float16)], axis=0)
        test_landmarks = np.append(test_landmarks.astype(np.float16), [(landmarks/255.0).astype(np.float16)], axis=0)
        test_count = 0
      else:
        test_images = np.append(test_images.astype(np.float16), [(greyscale_frame/255.0).astype(np.float16)], axis=0)
        test_masks = np.append(test_masks.astype(np.float16), [(mask/255.0).astype(np.float16)], axis=0)
        test_landmarks = np.append(test_landmarks.astype(np.float16), [(landmarks/255.0).astype(np.float16)], axis=0)

  # Imprimir la información de los datos
  print(f'ÍNDICE: {index}  VIDEO: {series[0]}  FOTOGRAMA: {series[1]}  CONJUNTO: {split}')

  # Mostrar el fotograma, la máscara, los puntos más importantes y las líneas del trazo
  frame[:,:,0] = cv2.addWeighted(mask.astype(np.uint8), 0.75, frame[:,:,0], 1, 0)
  for coord in coords:
    cv2.circle(frame, (int(coord[0]), int(coord[1])), 1, (0, 0, 255), -1)
  for _, point in points.iterrows():
    cv2.line(frame, (int(point.X1), int(point.Y1)), (int(point.X2), int(point.Y2)), (0, 255, 0), thickness=1)
  

# Crear directorio para guardar todos los arreglos como archivos
array_directory = os.path.join(os.getcwd(), 'Datasets4')
os.mkdir(array_directory)

# Guardar todos los arreglos de entrenamiento
np.save(os.path.join(array_directory, 'train_images.npy'), train_images.astype(np.float16))
np.save(os.path.join(array_directory, 'train_masks.npy'), train_masks.astype(np.float16))
np.save(os.path.join(array_directory, 'train_landmarks.npy'), train_landmarks.astype(np.float16))

# Guardar todos los arreglos de validación
np.save(os.path.join(array_directory, 'val_images.npy'), val_images.astype(np.float16))
np.save(os.path.join(array_directory, 'val_masks.npy'), val_masks.astype(np.float16))
np.save(os.path.join(array_directory, 'val_landmarks.npy'), val_landmarks.astype(np.float16))

# Guardar todos los arreglos de prueba
np.save(os.path.join(array_directory, 'test_images.npy'), test_images.astype(np.float16))
np.save(os.path.join(array_directory, 'test_masks.npy'), test_masks.astype(np.float16))
np.save(os.path.join(array_directory, 'test_landmarks.npy'), test_landmarks.astype(np.float16))

ÍNDICE: 15037  VIDEO: 0X699E3846EC61235F.avi  FOTOGRAMA: 57  CONJUNTO: TRAIN
ÍNDICE: 15038  VIDEO: 0X699F48578122088.avi  FOTOGRAMA: 62  CONJUNTO: TRAIN
ÍNDICE: 15039  VIDEO: 0X699F48578122088.avi  FOTOGRAMA: 76  CONJUNTO: TRAIN
ÍNDICE: 15040  VIDEO: 0X69A32C8856DC6879.avi  FOTOGRAMA: 107  CONJUNTO: TRAIN
ÍNDICE: 15041  VIDEO: 0X69A32C8856DC6879.avi  FOTOGRAMA: 122  CONJUNTO: TRAIN
ÍNDICE: 15042  VIDEO: 0X69A37D716D3C1EE2.avi  FOTOGRAMA: 37  CONJUNTO: TRAIN
ÍNDICE: 15043  VIDEO: 0X69A37D716D3C1EE2.avi  FOTOGRAMA: 53  CONJUNTO: TRAIN
ÍNDICE: 15044  VIDEO: 0X69A6D0FAEBA50027.avi  FOTOGRAMA: 112  CONJUNTO: TRAIN
ÍNDICE: 15045  VIDEO: 0X69A6D0FAEBA50027.avi  FOTOGRAMA: 131  CONJUNTO: TRAIN
ÍNDICE: 15046  VIDEO: 0X69A75DB4A1B8CA28.avi  FOTOGRAMA: 32  CONJUNTO: TRAIN
ÍNDICE: 15047  VIDEO: 0X69A75DB4A1B8CA28.avi  FOTOGRAMA: 43  CONJUNTO: TRAIN
ÍNDICE: 15048  VIDEO: 0X69AAEA04AAA0FB9D.avi  FOTOGRAMA: 106  CONJUNTO: TRAIN
ÍNDICE: 15049  VIDEO: 0X69AAEA04AAA0FB9D.avi  FOTOGRAMA: 122  CONJUNTO: T

### Validar que los arreglos se hayan guardado correctamente

In [54]:
# Imprimir las dimensiones de los archivos de entrenamiento
print('Imágenes de entrenamiento:', np.load(os.path.join(array_directory, 'train_images.npy')).shape)
print('Máscaras de entrenamiento:', np.load(os.path.join(array_directory, 'train_masks.npy')).shape)
print('Puntos de entrenamiento:  ', np.load(os.path.join(array_directory, 'train_landmarks.npy')).shape, '\n')

# Imprimir las dimensiones de los archvios de validación
print('Imágenes de validación:   ', np.load(os.path.join(array_directory, 'val_images.npy')).shape)
print('Máscaras de validación:   ', np.load(os.path.join(array_directory, 'val_masks.npy')).shape)
print('Puntos de validación:     ', np.load(os.path.join(array_directory, 'val_landmarks.npy')).shape, '\n')

# Imprimir las dimensiones de los archivos de prueba
print('Imágenes de prueba:       ', np.load(os.path.join(array_directory, 'test_images.npy')).shape)
print('Máscaras de prueba:       ', np.load(os.path.join(array_directory, 'test_masks.npy')).shape)
print('Puntos de prueba:         ', np.load(os.path.join(array_directory, 'test_landmarks.npy')).shape)

Imágenes de entrenamiento: (3636, 112, 112, 1)
Máscaras de entrenamiento: (3636, 112, 112, 1)
Puntos de entrenamiento:   (3636, 112, 112, 7) 

Imágenes de validación:    (666, 112, 112, 1)
Máscaras de validación:    (666, 112, 112, 1)
Puntos de validación:      (666, 112, 112, 7) 

Imágenes de prueba:        (708, 112, 112, 1)
Máscaras de prueba:        (708, 112, 112, 1)
Puntos de prueba:          (708, 112, 112, 7)


### OPCIONAL: Concatenar los nuevos arreglos a aquellos previamente almacenados



In [55]:
# #Definir el directorio con los archivos generados anteriormente
# complete_array_directory = os.path.join(os.getcwd(), 'CompleteDatasets')

# # Añadir los nuevos datos a los conjuntos existentes
# for file in os.listdir(complete_array_directory):
#   array = np.load(os.path.join(array_directory, file))
#   complete_array = np.load(os.path.join(complete_array_directory, file))
#   complete_array = np.concatenate((array, complete_array), axis=0)
#   np.save(os.path.join(complete_array_directory, file), complete_array)
#   print(f'El archivo {file} se actualizó con éxito')
#   print(f'  Dimensiones: {complete_array.shape}  Tipo: {complete_array.dtype}')